# 화장품 리뷰 형태소 분석
- 엑셀 파일을 읽는 데에 xlrd 모듈을 사용한다.
- `split_sentences()` 함수를 수정하여 빈 문자열에 대한 형태소 분석이 되지 않게 한다.

In [1]:
import xlrd
import re
from konlpy.tag import Komoran

KEY_NAMES = [
    "user_id", "channel", "option", "review", "membership"
]
MA_KEY_NAME = "review"


def read_documents(input_file_name):
    """주어진 이름의 엑셀 파일을 읽어서 JSON 라인 호환 리스트 객체를 만들어 돌려준다."""
    
    documents = []
    book = xlrd.open_workbook(input_file_name)
    sheet = book.sheet_by_index(0)
    
    for row_num in range(1, sheet.nrows):
        document = {}
        
        for col_num, key_name in enumerate(KEY_NAMES):
            value = sheet.cell_value(row_num, col_num)
            document[key_name] = value
            
        documents.append(document)

    return documents

In [2]:
def morph_anal_text(text, analyzer):
    """주어진 텍스트를 주어진 분석기로 형태소 분석하여 돌려준다."""
    
    text = text.strip()
    
    if not text:
        return []
    
    ma_text = []
    sentences = split_sentences(text)
    
    for sentence in sentences:
        ma_sentence = analyzer.pos(sentence)
        ma_text.append(ma_sentence)
        
    return ma_text
    
    
def split_sentences(text):
    """주어진 텍스트를 문장 단위로 분절하여 돌려준다."""
    
    all_sentences = []
    # 아래에서 line.strip을 line.strip()으로 수정
    # lines = [line for line in text.strip().splitlines() if line.strip]
    lines = [line for line in text.strip().splitlines() if line.strip()]
    
    # 빈 문장이 형태소 분석되지 않도록 함.
    for line in lines:
        line = line.strip()
        
        if not line:
            continue
            
        sentences = re.split("(?<=[.?!])\s+", line)
        all_sentences += sentences
        
    return all_sentences
    

In [3]:
def main():
    """화장품 리뷰 데이터에 대한 형태소 분석을 수행한다."""
    
    review_file_name = "reviews_after_del_similar.xlsx"
    documents = read_documents(review_file_name)
    komoran = Komoran()
    
    for document in documents:
        text = document[MA_KEY_NAME]
        ma_text = morph_anal_text(text, komoran)
        document[MA_KEY_NAME + "_ma"] = ma_text
    
    print(documents[:20])


main()

[{'user_id': 'kanade486', 'channel': 'MOBILE', 'option': '진주', 'review': '맨날 쓰던 것만 썼는데 진주가 은근히 평이 좋더라구요. 그래서 사봤는데 꽤 괜찮네요. 더운 날씨보단 가을 겨울 건성에게 더 좋을것 같아요. 촉촉함이랑 보습 미백 효과가 좋아요. 한장만 샀는데 날 추워지면 더 사려구요.', 'membership': '일반', 'review_ma': [[('맨날', 'MAG'), ('쓰', 'VV'), ('던', 'ETM'), ('것', 'NNB'), ('만', 'JX'), ('쓰', 'VV'), ('었', 'EP'), ('는데', 'EC'), ('진주', 'NNP'), ('가', 'JKS'), ('은근히', 'MAG'), ('평', 'NNB'), ('이', 'JKS'), ('좋', 'VA'), ('더라구요', 'EF'), ('.', 'SF')], [('그래서', 'MAJ'), ('사', 'VV'), ('아', 'EC'), ('보', 'VX'), ('았', 'EP'), ('는데', 'EC'), ('꽤', 'MAG'), ('괜찮', 'VA'), ('네요', 'EF'), ('.', 'SF')], [('덥', 'VA'), ('ㄴ', 'ETM'), ('날씨', 'NNG'), ('보다', 'JKB'), ('ㄴ', 'JX'), ('가', 'VV'), ('을', 'ETM'), ('겨울', 'NNG'), ('건성', 'NNG'), ('에게', 'JKB'), ('더', 'MAG'), ('좋', 'VA'), ('을', 'ETM'), ('것', 'NNB'), ('같', 'VA'), ('아요', 'EF'), ('.', 'SF')], [('촉촉', 'XR'), ('하', 'XSA'), ('ㅁ', 'ETN'), ('이랑', 'JC'), ('보습', 'NNG'), ('미백', 'NNP'), ('효과', 'NNG'), ('가', 'JKS'), ('좋', 'VA'), ('아요', 'EF'), ('.', 'SF')], [('한', 